<a href="https://colab.research.google.com/github/suresh-venkate/NLP_LLM/blob/main/Transformers/BERT/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Implementation - PyTorch



## Install Required Libraries

In [1]:
%%capture
pip install transformers datasets tokenizers

In [2]:
import os
from pathlib import Path
import re
import random
import tqdm
import itertools
import math
import numpy as np

import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

## Download dataset

Download cornell movie dialogs corpus ([Link](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html))

In [3]:
%%capture
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

## Load data into memory

In [4]:
### loading all data into memory
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

In [5]:
### split text
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [6]:
### generate question answer pairs
MAX_LEN = 64
pairs = []
ind = 0
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []

        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()
        second = lines_dic[ids[i+1]].strip()

        qa_pairs.append(' '.join(first.split()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)

## Wordpiece Tokenization

BERT employs a WordPiece tokenizer. To train the tokenizer, the BertWordPieceTokenizer from the transformer library is used with the steps below:



*   Saving the conversation text into multiple .txt files (with batch of N=10000)
*   Define BertWordPieceTokenizer with some parameters like **clean_text** to remove control characters, **handle_chinese_chars** to include spaces around Chinese characters, **stripe_accents** to remove accents and make é → e, ô → o, and **lowercase** to view capital and lowercase characters as equal.
*   Train the tokenizer based on the file path to .txt files with parameters like **vocab_size** defining the total number of tokens, **min_frequency** for minimum frequency for a pair of tokens to be merged, **special_tokens** defining a list of the special tokens that BERT uses, **limit_alphabet** for a maximum number of different characters, **workpieces_prefix** the prefix added to pieces of words (like ##ing).

In [7]:
### save data as txt file
os.mkdir('./data')
text_data = []
file_count = 0

In [8]:
for sample in tqdm.tqdm([x[0] for x in pairs]):
    text_data.append(sample)

    # once we hit the 10K mark, save to file
    if len(text_data) == 10000:
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 221616/221616 [00:00<00:00, 1281882.26it/s]


In [9]:
paths = [str(x) for x in Path('./data').glob('**/*.txt')]

The code below is used to train a custom tokenizer for a BERT model using the **BertWordPieceTokenizer** class. This tokenizer is designed to tokenize text data into subword tokens suitable for use with BERT-based models.

First, we define the tokenizer with the following parameters:

*   **clean_text**=True: This parameter specifies whether the text data should be cleaned before tokenization. When set to True, it performs text cleaning operations such as removing extra spaces and unwanted characters.
*   **handle_chinese_chars**=False: This parameter specifies whether the tokenizer should handle Chinese characters specially. If set to True, the tokenizer would treat Chinese characters as individual tokens; otherwise, it would treat them as parts of larger tokens.
*   **strip_accents**=False: This parameter specifies whether accents should be stripped from characters. When set to True, it removes accents from characters before tokenization
*   **lowercase**=True: This parameter specifies whether the text should be converted to lowercase before tokenization. When set to True, it converts all text to lowercase.

In [10]:
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

Then we train the tokenizer with the following parameters:

*   **files**=paths: This parameter specifies the list of file paths containing the text data used for training the tokenizer.
*   **vocab_size**=30_000: This parameter specifies the maximum size of the vocabulary (number of tokens) that the tokenizer will produce. In this case, the vocabulary size is set to 30,000 tokens.
* **min_frequency**=5: This parameter specifies the minimum frequency threshold for tokens to be included in the vocabulary. Tokens that occur less frequently than this threshold will not be included in the vocabulary.
* **limit_alphabet**=1000: This parameter specifies the maximum number of characters in the alphabet used for tokenization. It limits the characters that the tokenizer considers when building the vocabulary.
* **wordpieces_prefix**='##': This parameter specifies the prefix to be used for wordpieces. Wordpieces are subword tokens produced by the tokenizer. The ## prefix indicates that a token is a continuation of a previous token.
* **special_tokens**=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']: This parameter specifies a list of special tokens to be included in the vocabulary. These tokens have specific meanings within the BERT model architecture, such as padding token [PAD], classification token [CLS], separator token [SEP], mask token [MASK], and unknown token [UNK].

In [11]:
tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=5,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

os.mkdir('./bert-it-1')
# Save the tokenizer model to the directory './bert-it-1' with the prefix 'bert-it'
tokenizer.save_model('./bert-it-1', 'bert-it')
# Re-load the tokenizer from the previously saved model files located in the './bert-it-1' directory
# The argument './bert-it-1/bert-it-vocab.txt' specifies the path to the vocabulary file
# ('bert-it-vocab.txt') within the 'bert-it-1' directory.
# The local_files_only=True argument ensures that only locally available files are used for loading
# the tokenizer, meaning it won't attempt to download the model from the internet.
tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt', local_files_only=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1947: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


**Special Tokens in BERT**

* CLS stands for classification. It serves as the the Start of Sentence (SOS) and represent the meaning of the entire sentence.
* SEP serves as End of Sentence (EOS) and also the separation token between first and second sentences.
* PAD to be added into sentences so that all of them would be in equal length. During the training process, please note that the [PAD] token with id of 0 will not contribute to the gradient .
* MASK for word replacement during masked language prediction
* UNK serves as a replacement for token if it’s not being found in the tokenizer’s vocab.

## Class: BERTDataset

In [19]:
class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=64):

        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    def get_sent(self, index):
        '''return a random sentence pair and corresponding 'is_next' label'''
        t1, t2 = self.get_corpus_line(index)

        # negative or positive pair, for next sentence prediction
        if random.random() > 0.5:
            return t1, t2, 1 # Positive pair - Label is 1
        else:
            return t1, self.get_random_line(), 0 # Negative pair - Label is 0

    def get_corpus_line(self, item):
        '''return sentence pair'''
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        '''return random single sentence'''
        return self.lines[random.randrange(len(self.lines))][1]

    def random_word(self, sentence):
        """
        Performs the random replacement of tokens in each sentence using the given tokenizer object.
        """
        tokens = sentence.split()
        output_label = []
        output = []

        # 15% of the tokens would be replaced
        for i, token in enumerate(tokens):
            prob = random.random()

            # remove cls and sep token
            token_id = self.tokenizer(token)['input_ids'][1:-1]

            if prob < 0.15:
                prob /= 0.15

                # 80% chance change token to mask token
                if prob < 0.8:
                    for i in range(len(token_id)):
                        output.append(self.tokenizer.vocab['[MASK]'])

                # 10% chance change token to random token
                elif prob < 0.9:
                    for i in range(len(token_id)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))

                # 10% chance change token to current token
                else:
                    output.append(token_id)

                output_label.append(token_id)

            else:
                output.append(token_id)
                for i in range(len(token_id)):
                    output_label.append(0)

        # flattening
        # The following commands take a nested list as input, flattens it by removing one level of nesting,
        # and returns the flattened list as the output.
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
        assert len(output) == len(output_label) # Check whether output and output_label have the same length
        return output, output_label

    def __getitem__(self, item):

        # Step 1: get random sentence pair, either negative or positive (saved as is_next_label)
        t1, t2, is_next_label = self.get_sent(item)

        # Step 2: replace random words in sentence with mask / random words
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        # Step 3: Adding CLS and SEP tokens to the start and end of sentences
        # Adding PAD token for labels
        t1 = [self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]
        t2 = t2_random + [self.tokenizer.vocab['[SEP]']]
        t1_label = [self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]
        t2_label = t2_label + [self.tokenizer.vocab['[PAD]']]

        # Step 4: combine sentence 1 and 2 as one input
        # adding PAD tokens to make the sentence same length as seq_len
        # Explanation of below code:
        # segment_label: This line creates a list called segment_label that contains segment labels for tokens in the input sequences.
        # It combines segment labels for two sequences (t1 and t2) by creating a list of 1s (representing the first segment)
        # with a length equal to the number of tokens in t1, and a list of 2s (representing the second segment) with a length
        # equal to the number of tokens in t2. It then concatenates these two lists and truncates the result to the
        # desired sequence length (self.seq_len).
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        # bert_input: This line creates a list called bert_input by concatenating the tokens from t1 and t2,
        # and then truncates the result to the desired sequence length (self.seq_len). This list likely represents the input
        # tokens for the BERT model.
        bert_input = (t1 + t2)[:self.seq_len]
        # bert_label: This line creates a list called bert_label by concatenating the labels associated with tokens in t1 and t2,
        # and then truncates the result to the desired sequence length (self.seq_len). This list likely represents the labels
        # corresponding to the input tokens for the BERT model.
        bert_label = (t1_label + t2_label)[:self.seq_len]
        # padding: This line creates a list called padding containing [PAD] tokens (which typically represent padding tokens in
        # tokenized sequences) to fill the remaining space up to self.seq_len.
        padding = [self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
        # Padding the sequences: Finally, the code extends bert_input, bert_label, and segment_label lists with the padding
        # tokens to ensure that all sequences have the same length (self.seq_len). This padding step is crucial for batch processing,
        bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)

        output = {"bert_input": bert_input,
                  "bert_label": bert_label,
                  "segment_label": segment_label,
                  "is_next": is_next_label}

        return {key: torch.tensor(value) for key, value in output.items()}

In [20]:
train_data = BERTDataset(
   pairs, seq_len=MAX_LEN, tokenizer=tokenizer)

In [24]:
train_data[1]

{'bert_input': tensor([    1,   303,    15,     3,   515,   184,    11,    43,   672,   231,
         15149,   295,  7732,    96,   242,    15,     3,   173,    11,    58,
           459,   231,   146,    17,     2,  1884,    17,   493,  1884,    17,
            40,     3,     3,     2,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'bert_label': tensor([   0,    0,    0,   48,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,  270,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, 3585,   17,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 'segment_label': t